In [1]:
import sys
import importlib
import os
# make the module available from the src directory
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
data_dir = "../datasets/artificial_data/3d_shapes"

def create_list():
    pairs = []
    all_files = os.listdir(data_dir)
    for filename in all_files:
        label = int(filename.split("_").pop().replace(".pt", ""))
        label_dict = { "size": label }
        file_path = os.path.join(data_dir, filename)
        pairs.append({"image": file_path, "label": label_dict})
    return pairs

pairs = create_list()
print(pairs)

[{'image': '../datasets/artificial_data/3d_shapes/volume_40_label_15.pt', 'label': {'size': 15}}, {'image': '../datasets/artificial_data/3d_shapes/volume_0_label_29.pt', 'label': {'size': 29}}, {'image': '../datasets/artificial_data/3d_shapes/volume_10_label_26.pt', 'label': {'size': 26}}, {'image': '../datasets/artificial_data/3d_shapes/volume_15_label_13.pt', 'label': {'size': 13}}, {'image': '../datasets/artificial_data/3d_shapes/volume_39_label_15.pt', 'label': {'size': 15}}, {'image': '../datasets/artificial_data/3d_shapes/volume_37_label_16.pt', 'label': {'size': 16}}, {'image': '../datasets/artificial_data/3d_shapes/volume_38_label_18.pt', 'label': {'size': 18}}, {'image': '../datasets/artificial_data/3d_shapes/volume_6_label_17.pt', 'label': {'size': 17}}, {'image': '../datasets/artificial_data/3d_shapes/volume_9_label_18.pt', 'label': {'size': 18}}, {'image': '../datasets/artificial_data/3d_shapes/volume_32_label_22.pt', 'label': {'size': 22}}, {'image': '../datasets/artificia

In [5]:
from src.transformers.TensorLoaderTransfomer import TensorLoader
from monai.transforms import Compose, LoadImaged, EnsureChannelFirstD, ScaleIntensityd, ToTensord, ResizeD, Lambdad, NormalizeIntensityD

from src.transformers.SliceTransformer import SliceAggregateTransform
importlib.reload(sys.modules['src.transformers.SliceTransformer'])

importlib.reload(sys.modules['src.transformers.TensorLoaderTransfomer'])
transforms = [
    TensorLoader(keys=["image"]),
    SliceAggregateTransform(keys=["image"], slices=9),
    ScaleIntensityd(keys=["image"], minv=0.0, maxv=1.0, factor=1.0),
    NormalizeIntensityD(keys=["image"], nonzero=True, channel_wise=True),
    ]

In [6]:
from src.dataLoaders.NiftiDataLoader2 import NiftiDataLoader
importlib.reload(sys.modules['src.dataLoaders.NiftiDataLoader2'])

<module 'src.dataLoaders.NiftiDataLoader2' from '/Users/askelundsgaard/Documents/datalogi/6-semester/Bachelor/XAI-in-Medical-Image-Analysis/src/dataLoaders/NiftiDataLoader2.py'>

In [7]:
dataset = 'niftiShort'
data_path = '../datasets/nifti/'
data_dir = f'{data_path}{dataset}'

# data_dir = 'C:/Users/askel/Downloads/NIFTY/NIFTY/'
# max = 8876
dim = 384
data_loader = NiftiDataLoader(data_dir=data_dir, 
                              batch_size=32,
                              spatial_resize=(dim, dim, dim),
                              cache_rate=0.5, 
                              replace_rate=1,
                              transforms=transforms,
                              data_list=pairs,
                              )
# data_loader.load_data(visit_no=visit, subset_size=2058, cache="standard")
# data_loader.load_data(subset_size=84, cache="standard")
#data_loader.load_data(subset_size=4400, cache="standard")
# data_loader.load_data(subset_size=8875, cache="standard")

data_loader.load_data(cache="standard")

File /Users/askelundsgaard/Documents/datalogi/6-semester/Bachelor/XAI-in-Medical-Image-Analysis/src/dataLoaders/saved_data_lists/data_list.pkl does not exist.
Data list loaded: False
Using custom sampler: False
Subset size: 50


Loading dataset: 100%|██████████| 5/5 [00:00<00:00, 72.53it/s]

batch_size: 32


In [8]:
from src.models.medical_models.medical_resnet import MedicalResNetModel
importlib.reload(sys.modules['src.models.medical_models.base_medical'])
importlib.reload(sys.modules['src.models.medical_models.medical_resnet'])


from src.models.medical_models.monai_resnet import MonaiMedicalResNet
importlib.reload(sys.modules['src.models.medical_models.monai_resnet'])

from src.models.medical_models.combined_medical import MedicalCombinedResNetModel
importlib.reload(sys.modules['src.models.medical_models.combined_medical'])

<module 'src.models.medical_models.monai_resnet' from '/Users/askelundsgaard/Documents/datalogi/6-semester/Bachelor/XAI-in-Medical-Image-Analysis/src/models/medical_models/monai_resnet.py'>

In [12]:
fusion_model = MedicalCombinedResNetModel(
    num_epochs=100,
    learning_rate=0.001,
    weight_decay=1e-4,
    data_loader=data_loader,
    dropout_rate=0.1,
    depth=18
    #pretrained=False
)

fusion_model.train()
fusion_model.evaluate()

Data loader train loader:  2 40
Image spatial dimensions:  torch.Size([32, 9, 256, 256])
Number of input channels:  9
gpu:  mps:0
Is cuda available:  False mps
Number of training images: 40
Epoch 1/100, Train Loss: 411.1060485839844, Val Loss: 311.44537353515625, R^2 Score: -15.708450317382812
Model saved at /Users/askelundsgaard/Documents/datalogi/6-semester/Bachelor/XAI-in-Medical-Image-Analysis/src/models/medical_models/saved_models/MedicalCombinedResNetModel_18_40_height_256_epoch_1_val_311.45_r2_311.45.pth
Epoch 2/100, Train Loss: 246.43927001953125, Val Loss: 287.8081359863281, R^2 Score: -14.44035816192627
Model saved at /Users/askelundsgaard/Documents/datalogi/6-semester/Bachelor/XAI-in-Medical-Image-Analysis/src/models/medical_models/saved_models/MedicalCombinedResNetModel_18_40_height_256_epoch_2_val_287.81_r2_287.81.pth
Epoch 3/100, Train Loss: 136.5634307861328, Val Loss: 30.897916793823242, R^2 Score: -0.6576143503189087
Model saved at /Users/askelundsgaard/Documents/datal

In [ ]:
# Create a model instance
model = MedicalResNetModel(
    num_epochs=100,
    learning_rate=0.001,
    weight_decay=1e-4,
    data_loader=data_loader,
    dropout_rate=0.1,
    depth=18
    #pretrained=False
)

# model.load_model('MedicalResNetModel_18_6212_epoch_38.pth')
model.train()
model.evaluate()

In [9]:
# Create a model instance
model = MedicalResNetModel(
    num_epochs=100,
    learning_rate=0.001,
    weight_decay=1e-4,
    data_loader=data_loader,
    dropout_rate=0.1,
    depth=18
    #pretrained=False
)

# model.load_model('MedicalResNetModel_18_6212_epoch_38.pth')
model.train()
model.evaluate()

Data loader train loader:  2 40
Image spatial dimensions:  torch.Size([32, 9, 256, 256])
Number of input channels:  9
gpu:  mps:0
Is cuda available:  False mps
Number of training images: 40
Epoch 1/100, Train Loss: 442.7793273925781, Val Loss: 425.1249084472656, R^2 Score: -21.80714225769043
Model saved at /Users/askelundsgaard/Documents/datalogi/6-semester/Bachelor/XAI-in-Medical-Image-Analysis/src/models/medical_models/saved_models/MedicalResNetModel_18_40_height_256_epoch_1_val_425.12_r2_425.12.pth
Epoch 2/100, Train Loss: 409.0142364501953, Val Loss: 267.2445983886719, R^2 Score: -13.337162971496582
Model saved at /Users/askelundsgaard/Documents/datalogi/6-semester/Bachelor/XAI-in-Medical-Image-Analysis/src/models/medical_models/saved_models/MedicalResNetModel_18_40_height_256_epoch_2_val_267.24_r2_267.24.pth
Epoch 3/100, Train Loss: 303.0432434082031, Val Loss: 241.3467254638672, R^2 Score: -11.94779109954834
Model saved at /Users/askelundsgaard/Documents/datalogi/6-semester/Bache